In [1]:
import re
from bs4 import BeautifulSoup

In [6]:
fname = 'wholeDictionaryFromDoc.htm'

with open(fname, 'r') as f:
    global_soup = BeautifulSoup(f.read(), 'html.parser')
    to_soup = str(global_soup.find('div', class_ = 'WordSection2')) + str(global_soup.find('div', class_ = 'WordSection3'))
    global_soup = BeautifulSoup(to_soup, 'html.parser')
paragraphs = global_soup.find_all('p')

open('wholeDictionaryFromDoc_dictOnly.htm', 'w', encoding='utf-8').write(str(global_soup))
open('plainDictText_div23.txt', 'w', encoding='utf-8').write(global_soup.text)

1286895

In [8]:
rawParagraphsList = [re.sub('\n', ' ', str(p)) for p in paragraphs]
rawParagraphsStr = '\n'.join(rawParagraphsList)

In [13]:
with open('rawParagraphsStr.htm', 'w', encoding='utf-8') as f:
    f.write(rawParagraphsStr)

In [15]:
## произошел ручной эдитинг пары абзацев, которые плохо соединились ввиду
## 1) плохого распознания исходного документа
## 2) соединения двух <div> в один

## вручную втч. были удалены титульные абзацы (было быстрее, чем текстом)

## 'rawParagraphsStr.htm' = основной рабочий файл теперь, там решена проблема начальных ЛЪ_ 
## сперва надо будет чистить от тэгов

In [5]:
with open('rawParagraphsStr.htm', 'r', encoding='utf-8') as f:
    plainDict = f.read()
    plainDict = re.sub('</?o:p>', '', plainDict)
    plainDict = re.sub('</?s?pa?n?[^>]*>', '', plainDict)
    
    plainDict = re.sub('</?i[^>]*>', '_', plainDict)
    plainDict = re.sub('</?b[^>]*>', '*', plainDict)
    
    
    ## назализация: потом надо будет заменить на надстрочный "н"
    plainDict = re.sub('<sup>[нН].?</sup>', 'ᴴ', plainDict)
    
    plainDict = re.sub('\*\*', '', plainDict)
    plainDict = re.sub('__', '', plainDict)
    
    plainDict = re.sub('_\.', '._', plainDict)
    
    with open('rawParagraphsCleared.htm', 'w', encoding='utf-8') as fw:
        fw.write(plainDict)
    #plainDict = re.sub('<sup>Н</sup>')
    
    # надо подумать о неразрывных пробелах и о лишних пробелах

In [10]:
with open('rawParagraphsCleared.htm', 'r', encoding='utf-8') as f:
    plainDict = f.read()
    
regLatin = re.compile('_лат\._ ([^\(\n;\d_]+)')
latins = regLatin.findall(plainDict)
print(len(latins))
print(latins)

40
['Trifolium repens', 'Condoliulus arvense', 'ARCHANGELICA OFFICINALIS', 'Caloglossum ofride', 'Primula verisl ', 'Salvia', 'Anetrum lapg', 'Menthac longlfaem', 'Fumoria Sehleicheri ', 'Asperula hamufusa', 'stachy palustris', 'Astemia axsinthum', '*хамесциациум* ', 'Peucejanum nuthenicum ', 'Seseli franscaucastce ', 'lapsano media', 'Pactulus glomepafa', 'Stellarta media', 'Carum carvi', 'Vitia', 'Thumus seppillum l ', 'Ziziphora Sp ', 'Hyoscyamus niger', 'Plantago major', 'Galium aparine', 'Zathyrus minuatus', 'Chenopodium Sp', 'Vonatcum labelianum', 'Pragnuges Communis', 'Anthriyscus nemoralis', 'Cenfiana cructafa', 'chocnophyclum', 'Origanum vulgare', 'Cubus saxatitis', 'loricera xylosfeum', 'Rapistrum rugogum ', 'Galega arientalis', 'Rumex acelosella', 'inonotus obbiglus ', 'Malka neglecta']


In [39]:
with open('listOfLatinNamesInDict.txt', 'w', encoding='utf-8') as fl:
    fl.write('\n'.join(latins))

In [10]:
plainDict = re.sub(' ', ' ', plainDict)
plainDict = re.sub('\r', '', plainDict)
plainDict = re.sub('\n ', '\n', plainDict)
plainDict = re.sub('\n\* ', '\n*', plainDict)
plainDict = re.sub('<sup> </sup>', ' ', plainDict)
plainDict = re.sub(r'<sup>(\d)</sup>', r'№\1', plainDict)
plainDict = re.sub('\*\*', '', plainDict)

with open('rawParagraphsCleared_v2.txt', 'w', encoding='utf-8') as fv2:
    fv2.write(plainDict)

In [ ]:
## <sup>н</sup> = 
## <sup>Н</sup> = 
## </?i> = _
## </?b> = *
## <sup>.</sup> = 

## ̄надстрочная палка; символ стоит после последнего символа лигатуры, но перед ударением:
#### ЛЪ̄
#### а̄'


## symbols_pivot
## E:
### 1. E(\s) = Е 
### 2. E(\S) = Л̄

## swap(_I, I_)

## "(\d\))([а-яёА-ЯЁa-zA-Z])", r"\1 \2"

In [24]:
with open('rawParagraphsCleared_v2.txt', 'r', encoding='utf-8') as fv2:
    dictPlain = fv2.read()

with open('symbols_pivot.csv', 'r', encoding='utf-8') as ft:
    rawTable = re.sub('\r', '', ft.read())
    table = [x.split(';') for x in rawTable.split('\n') if len(x.split(';')) == 2]

In [25]:
for pair in table:
    dictPlain = re.sub(pair[0], pair[1], dictPlain)
    
dictPlain = re.sub('̄I', 'Ī', dictPlain)

In [26]:
with open('rawParagraphsClearedAndReplaced.txt', 'w', encoding='utf-8') as fv3:
    fv3.write(dictPlain)

In [41]:
with open('rawParagraphsClearedAndReplaced.txt', 'r', encoding='utf-8') as fv3:
    dictPlain = fv3.read()
    testSymbols = re.sub('_лат\._ \S+ \S+', '', dictPlain)
    checkset = set(re.findall('[^а-яА-ЯI_\*,\.№\) \n]', testSymbols))

In [ ]:
#Вручную заменил все инвалидные (то есть побитые при операциях замены) тэги

In [34]:
checkset

{'!',
 "'",
 '(',
 '-',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '\\',
 'e',
 'g',
 'i',
 'l',
 'm',
 'o',
 's',
 'u',
 'x',
 'y',
 '«',
 '»',
 '̄',
 'ё',
 'ᴴ',
 '–',
 '…',
 '◊'}

In [11]:
brokenLatin = regLatin.findall(dictPlain)

In [12]:
brokenLatin

["Ц̄ц̄ifolium ц̄eц̄eи's",
 "К̄oи'doliulus л̄ц̄veи'se",
 "С̄И'К̄лъ̄С̄Х̄GЛ̄цIĪК̄С̄ х̄л̄лĪКĪХ̄С̄цIĪи'",
 'К̄л̄loglossum ofц̄ide',
 'Pц̄imulл̄ veц̄isl ',
 "и'л̄lviл̄",
 "С̄и'eӣц̄um lл̄ц̄g",
 "ЦĪeи'ӣhл̄лъ̄ loи'glfл̄em",
 "л̄umoц̄iл̄ и'ehleiлъ̄heц̄i ",
 'С̄sц̄eц̄ulл̄ hл̄mufusл̄',
 'sӣл̄лъ̄hy ц̄л̄lusӣц̄is',
 "С̄sӣemiл̄ л̄xsiи'ӣhum",
 '*хамесциациум* ',
 "Peuлъ̄eх̄л̄и'um и'uӣheи'iлъ̄um ",
 "и'eseli fц̄л̄и'sлъ̄л̄uлъ̄л̄sӣлъ̄e ",
 "lл̄ц̄sл̄и'o mediл̄",
 'Pл̄лъ̄ӣulus glomeц̄л̄fл̄',
 "и'ӣellл̄ц̄ӣл̄ mediл̄",
 'К̄л̄ц̄um лъ̄л̄ц̄vi',
 'Ч̄iӣiл̄',
 'Ц̄humus seц̄ц̄illum l ',
 "Ӯiziц̄hoц̄л̄ и'ц̄ ",
 "лъ̄yosлъ̄yл̄mus и'igeц̄",
 "Plл̄и'ӣл̄go mл̄х̄oц̄",
 "Gл̄lium л̄ц̄л̄ц̄iи'e",
 "Ӯл̄ӣhyц̄us miи'uл̄ӣus",
 "К̄heи'oц̄odium и'ц̄",
 "Ч̄oи'л̄ӣлъ̄um lл̄beliл̄и'um",
 "Pц̄л̄gи'uges К̄ommuи'is",
 "С̄и'ӣhц̄iysлъ̄us и'emoц̄л̄lis",
 "К̄eи'fiл̄и'л̄ лъ̄ц̄uлъ̄ӣл̄fл̄",
 "лъ̄hoлъ̄и'oц̄hyлъ̄lum",
 "х̄ц̄igл̄и'um vulgл̄ц̄e",
 'К̄ubus sл̄xл̄ӣiӣis',
 'loц̄iлъ̄eц̄л̄ xylosfeum',
 "И'л̄ц̄isӣц̄um ц̄ugogum ",

In [13]:
# точно ли "сломанным" латинским названиям соответствует лишь один фрагмент всего документа
def isItSafeToReSubLatins(brokenLatin=brokenLatin, plainDict=plainDict):
    for bl in brokenLatin:
        bl = re.sub('\*', '\\\*', bl)
        if len(re.findall(bl, plainDict)) > 1:
            return False
    return True

In [14]:
isItSafeToReSubLatins()

True

In [15]:
with open('listOfLatinNamesInDict.txt', 'r', encoding='utf-8') as flt:
    trueLatNames = re.sub('\r', '', flt.read()).split('\n')
    for index, item in enumerate(trueLatNames):
        bl = brokenLatin[index]
        bl = re.sub('\*', '\\\*', bl)
        dictPlain = re.sub(bl, item, dictPlain)

In [16]:
with open('rawParagraphsClearedAndReplacedFinal.txt', 'w', encoding='utf-8') as fv3:
    fv3.write(dictPlain)

In [52]:
with open('rawParagraphsClearedAndReplacedFinal.txt', 'r', encoding='utf-8') as fv3:
    dictPlain = fv3.read()
    
dictPlain = re.sub('\(-\*', '(*-', dictPlain)
dictPlain = re.sub("'ᴴ", "ᴴ'", dictPlain)

In [ ]:
# вручную добавил разметку на главы П и С (?), там почему-то слетели b-тэги

In [96]:
with open('rawParagraphsClearedAndReplacedFinal.txt', 'r', encoding='utf-8') as fv3:
    dictPlain = fv3.read()
    

In [97]:
paragraphs = dictPlain.split('\n')

In [98]:
len(paragraphs)

7132

In [19]:
import re

In [118]:
# импорт составленной таблицы букво-фонемных соответствий
with open('ipaChart.txt', 'r', encoding='utf-8') as ipaF:
    rawTable = re.sub('\r', '', ipaF.read())
    ipaTable = [x.split(';') for x in rawTable.split('\n') if len(x.split(';')) == 2]
    ipaTable = sorted(ipaTable, key=lambda x: len(x[0]), reverse=True)
    
print(ipaTable)

[['КĪВ', 'kʼʷː'], ['КЪIВ', 'q͡χ’ʷ'], ['КЬIВ', 't͡ɬʼʷ'], ['ЛЪ̄В', 'ɬʷː'], ['ЛЪIВ', 't͡ɬʷ'], ['ХЪIВ', 'q͡χʷ'], ['ЦĪВ', 't͡s’ʷː'], ['ЧĪВ', 't͡ʃ’ʷː'], ['Аᴴ̄', 'ãː'], ['ГЬВ', 'hʷ'], ['ГЪВ', 'ʁʷ'], ['ГIВ', 'ʕʷ'], ['Еᴴ̄', 'ẽː'], ['ДЖВ', 'd͡ʒʷ'], ['Иᴴ̄', 'ĩː'], ['К̄В', 'kʷː'], ['КЪВ', 'q͡χ’ʷː'], ['КЬВ', 't͡ɬʼʷː'], ['КIВ', 'k’ʷ'], ['КĪ', 'kʼː'], ['КЪI', 'q͡χ’'], ['КЬI', 't͡ɬʼ'], ['ЛЪ̄', 'ɬː'], ['ЛЪI', 't͡ɬ'], ['ЛЪВ', 'ɬʷ'], ['ЛIВ', 't͡ɬʷː'], ['Оᴴ̄', 'õ'], ['С̄В', 'sʷː'], ['ТIВ', 't’ʷ'], ['Уᴴ̄', 'ũː'], ['Х̄В', 'χʷː'], ['ХЪВ', 'q͡χʷː'], ['ХЪI', 'q͡χ'], ['ХЬВ', 'xʷː'], ['ХIВ', 'ћʷ'], ['Ц̄В', 't͡sʷː'], ['ЦĪ', 't͡s’ː'], ['Ч̄В', 't͡ʃ’ʷː'], ['ЧIВ', 't͡ʃ’ʷ'], ['ЧĪ', 't͡ʃ’ː'], ['Эᴴ̄', 'ẽː'], ['Аᴴ', 'ã'], ['А̄', 'aː'], ['ГВ', 'gʷ'], ['ГЬ', 'h'], ['ГЪ', 'ʁ'], ['ГI', 'ʕ'], ['ДВ', 'dʷ'], ['Еᴴ', 'ẽ'], ['Е̄', 'eː'], ['ЖВ', 'ʒʷ'], ['ДЖ', 'd͡ʒ'], ['ЗВ', 'zʷ'], ['Иᴴ', 'ĩ'], ['Ӣ', 'iː'], ['КВ', 'kʷ'], ['К̄', 'kː'], ['КЪ', 'q͡χ’ː'], ['КЬ', 't͡ɬʼː'], ['КI', 'k’'], ['ЛЪ', 'ɬ'], ['Л̄', 'lː'], ['ЛI', '

In [129]:
def phonetizerPlus(word, ipaTable=ipaTable, removeSlash=True):
    #print(word)
    word = re.sub('- ?', '', word.strip())
    word = re.sub('\([^\)][\)$]', '', word)
    #classMarker = re.findall('.?/[^/]{1/.?',
    phonWord = word
    for pair in ipaTable:
        cyr = pair[0]
        repl = '-' + pair[1] + '-'
        phonWord = re.sub(cyr, repl, phonWord)
        #print(cyr, repl, phonWord)
    #print(phonWord)
    phonWord = re.sub('--+', '-', phonWord)
    phonWord = re.sub("-'-", "'-", phonWord)
    phonWord = re.sub("- ", ' ', phonWord)
    phonWord = re.sub(" -", ' ', phonWord)
    if removeSlash:
        phonWord = re.sub("-/-", '-', phonWord)
        phonWord = re.sub("-'/", "'-", phonWord)
    return re.sub('i', 'I', word.lower()), re.sub('--+', '-', phonWord).strip('-')

In [136]:
globalList = list()
for rawArticle in paragraphs:
    
    rawArticle = re.sub('№(\d)', '', rawArticle)
    #print(rawArticle)
    try:
        boldSpans = re.findall('\*([^\*]+)\*', rawArticle)
        rawLemmaFirst = boldSpans[0]
        if len(boldSpans) > 1:
            if len(re.findall('[А-Я]', boldSpans[1])) > 2:
                rawLemmaFirst = ''.join(rawArticle.split('*', maxsplit=4)[:4])
                offset = 4
            else:
                offset = 2
        else:
            offset = 2
    except:
        print(rawArticle)
    #print(rawLemmaFirst)
    
    
     
    if re.search(':', rawLemmaFirst):
        rawLemmaFirst = rawLemmaFirst.split(':')[0]
    
    rawLemma = re.sub(' I+', '', rawLemmaFirst)
    
    #rawLemmas = [x.strip() for x in rawLemma.split('//')]
    
    
    simpleGrammar = re.search('\([^\)]+\)', rawLemmaFirst)
    if simpleGrammar:
        morphology = simpleGrammar.group()
        rawLemma = re.split(morphology, rawLemma)[0]
        rawLemmaFirst = rawLemma
        if '_' in morphology:
            morphology = re.sub('_[^_]+_', '', morphology)
            if len(morphology) < 5:
                morphology = ''
                definition = rawArticle[len(rawLemmaFirst)+offset:]
        
    else:    
        rawGrammar = re.search('\(([^\)]*\*[^\)]+)\)', rawArticle)
    
        if rawGrammar:
            rawMorphology = rawGrammar.group()
            morphology = re.sub('_[^_]+_', '', rawMorphology)
            if len(morphology) < 5:
                morphology = ''
                definition = rawArticle[len(rawLemmaFirst)+offset:]
            else:
                morphology = re.sub('\*', '', morphology) 
                morphology = '(' + morphology.strip(')(').strip(',') + ')'
                definition = rawArticle.split(rawMorphology)[1].strip()
        #print(1, morphology)
        else:
        
            morphology = ''
            definition = rawArticle[len(rawLemmaFirst)+offset:]
        
    rawLemmas = [x.strip() for x in re.split(' ?[ /][ /] ?', rawLemma)]
    for rawLemma in rawLemmas:
        #print(morphology)
        if len(rawLemma) >= 2:
            articleDict = dict()
            lemma_source, lemma_ipa = phonetizerPlus(rawLemma)
            articleDict['morphology'] = morphology
            articleDict['lemma_source'] = lemma_source
            articleDict['lemma_ipa'] = lemma_ipa
            articleDict['definition'] = definition
            globalList.append(articleDict)

In [137]:
len(globalList)

8003

In [138]:
with open('akhvakhDBv3.csv', 'w', encoding='utf-8') as csvfile:
    csvfile.write('word_id' + '\t' + '\t'.join((list(articleDict.keys()))))
    for index, article in enumerate(globalList, start=1):
        csvfile.write('\n')
        row = [str(index)]
        for key in article:
            row.append(article[key])
        rowS = '\t'.join(row)
        csvfile.write(rowS)